In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Conversation Memory

In langrila, message and response are serializable, so you can store the conversation history whereever the json object can be stored. Currently support: 

- InMemory
- Pickle (local file)
- JSON (local file)
- S3
- Azure Cosmos DB

If you don't specify the conversation memory for sub-agent, `InMemoryConversationMemory` is internally used in default, which is not persisted automatically within a sub-agent. Each sub-agent always need own conversation memory because the state within the multi-agent is kept or updated based on the conversation history and response schema if specified. Please be aware of using internal memory even if you don't specify the conversation memory for the sub-agent.

However, agent has `use_last_n_turns` argument to control how many conversation history we consider, default value is -1 which means all conversation history within the agent is used. The `turn` in this place is the sequence from the user input to the final answer of the agent including internal looped conversation in the agent. If 0, no conversation history is used. Even you specified `use_last_n_turns`, entire conversation memory is stored for debugging, not overwritten. `use_last_n_turns` parameter is used to extract the specified turn history from the entire conversation history.

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import (
    Agent,
    InMemoryConversationMemory,
    JSONConversationMemory,
    PickleConversationMemory,
)
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.memory.cosmos import CosmosConversationMemory
from langrila.memory.s3 import S3ConversationMemory
from langrila.openai import OpenAIClient

# Instantiate client wrapper

In [4]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(api_key_env_name="GEMINI_API_KEY")

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(api_key_env_name="ANTHROPIC_API_KEY")

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

# Define tools

Using dummy tools

In [5]:
import random
from enum import Enum

from pydantic import BaseModel, Field


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class EqualizerSettings(BaseModel):
    hz_100: int = Field(0, ge=-12, le=12, description="The 100 Hz band in dB.")
    hz_200: int = Field(0, ge=-12, le=12, description="The 200 Hz band in dB.")
    hz_400: int = Field(0, ge=-12, le=12, description="The 400 Hz band in dB.")
    hz_800: int = Field(0, ge=-12, le=12, description="The 800 Hz band in dB.")
    hz_1600: int = Field(0, ge=-12, le=12, description="The 1600 Hz band in dB.")
    hz_3200: int = Field(0, ge=-12, le=12, description="The 3200 Hz band in dB.")
    hz_6400: int = Field(0, ge=-12, le=12, description="The 4000 Hz band in dB.")


def start_music(genre: MusicGenre) -> str:
    """
    Start playing a random music with the given the genre.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to play.

    Returns
    ----------
    str
        A message indicating that the music has started.
    """

    bpms = {
        MusicGenre.rock: 150,
        MusicGenre.pop: 110,
        MusicGenre.jazz: 120,
        MusicGenre.classical: 76,
        MusicGenre.hip_hop: 100,
    }

    return (
        f"Starting music! Genre: {genre}, "
        f"Volume: {random.uniform(0.0, 1.0)}, "
        f"BPM: {bpms[genre]}"
    )


def change_music_volume(volume: float) -> str:
    """
    Change the music volume.

    Parameters
    ----------
    volume : float
        The new volume to set. It should be between 0.0 and 1.0.

    Returns
    ----------
    str
        A message indicating that the volume has been changed.
    """
    return f"Changing volume to {volume}"


def change_music(genre: MusicGenre) -> str:
    """
    Change the music.

    Parameters
    ----------
    genre : MusicGenre
        The genre of the music to change to.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    bpm_ranges = {
        MusicGenre.rock: (120, 180),
        MusicGenre.pop: (80, 140),
        MusicGenre.jazz: (70, 120),
        MusicGenre.classical: (70, 100),
        MusicGenre.hip_hop: (80, 120),
    }

    new_bpm = random.randint(bpm_ranges[genre][0], bpm_ranges[genre][1])

    return f"Turning to another music! Genre: {genre}, BPM: {new_bpm}"


def change_equalizer_settings(settings: EqualizerSettings) -> str:
    """
    Change the equalizer settings of the music.

    Parameters
    ----------
    settings : EqualizerSettings
        The new equalizer settings to set.

    Returns
    ----------
    str
        A message indicating that the equalizer settings have been changed.
    """
    return f"Changing equalizer settings to {settings.model_dump(exclude_none=True, exclude_unset=True)}"


def turn_light_on() -> str:
    """
    Turn the lights on.

    Returns
    ----------
    str
        A message indicating that the lights are turning on.
    """
    brightness = random.uniform(0.5, 1.0)
    return "Lights are now on! Brightness: {:.2f}".format(brightness)


def dim_lights(brightness: float) -> bool:
    """
    Dim the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully dimmed.
    """
    return f"Lights are now set to {brightness}"


def power_disco_ball(power: bool) -> bool:
    """
    Powers the spinning dissco ball on or off.

    Parameters
    ----------
    power : bool
        Whether to power the disco ball or not.

    Returns
    ----------
    bool
        Whether the disco ball is spinning or not.
    """
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def adjust_lights(brightness: float) -> bool:
    """
    Adjust the brightness of the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully brightened.
    """
    return f"Lights are now set to {brightness}"

# Multi-Agent

## Response schema

In [6]:
class DiscoBallSchema(BaseModel):
    power: bool = Field(..., description="power state of the disco ball.")


class MusicPropertySchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music.",
    )
    volume: float = Field(
        ...,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    bpm: int = Field(
        ...,
        description="The beats per minute of the music.",
    )


class ChangedMusicSchema(BaseModel):
    genre: MusicGenre = Field(
        None,
        description="The genre of music.",
    )
    volume: float = Field(
        None,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    bpm: int = Field(
        None,
        description="The beats per minute of the music.",
        ge=50,
        le=200,
    )


class MusicSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music.",
    )
    volume: float = Field(
        ...,
        description="The volume level of the music.",
        ge=0,
        le=1,
    )
    bpm: int = Field(
        ...,
        description="The beats per minute of the music.",
    )
    equalizer_settings: EqualizerSettings = Field(
        ...,
        description="The equalizer settings.",
    )


class LightsSchema(BaseModel):
    brightness: float = Field(
        ...,
        description="The brightness level to set the lights to.",
        ge=0,
        le=1,
    )


class ResponseSchema(BaseModel):
    disco_ball: DiscoBallSchema = Field(..., description="The disco ball settings.")
    music: MusicSchema = Field(..., description="The music settings.")
    lights: LightsSchema = Field(..., description="The lights settings.")

## Task specific agent.

In [7]:
from langrila import SystemPrompt

lights_agent = Agent(
    client=google_dev_client,
    model="gemini-1.5-flash-002",
    temperature=0.0,
    tools=[turn_light_on, adjust_lights],
    response_schema_as_tool=LightsSchema,
    use_last_n_turns=1,  # Use the last turn for the state of the lights.
)

disco_ball_agent = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    tools=[power_disco_ball],
    max_tokens=500,
    response_schema_as_tool=DiscoBallSchema,
    use_last_n_turns=1,  # Use the last turn for the state of the disco ball.
)

music_control_agent = Agent(
    client=google_dev_client,
    model="gemini-1.5-flash-002",
    temperature=0.0,
    tools=[start_music, change_music, change_music_volume],
    planning=True,
    response_schema_as_tool=MusicPropertySchema,
    use_last_n_turns=1,  # Use the last turn for the state of the music control.
    system_instruction=SystemPrompt(
        contents="You can start the music, or change the music genre and volume."
    ),
)

equalizer_control_agent = Agent(
    client=google_dev_client,
    model="gemini-1.5-flash-002",
    temperature=0.0,
    tools=[change_equalizer_settings],
    planning=True,
    response_schema_as_tool=EqualizerSettings,
    use_last_n_turns=1,  # Use the last turn for the state of the music control.
    system_instruction=SystemPrompt(contents="You can change the equalizer settings."),
)

## Orchestrator agent

In [8]:
# Orchestrator agent as a sub-agent
music_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    subagents=[music_control_agent, equalizer_control_agent],
    max_tokens=500,
    response_schema_as_tool=MusicSchema,
    use_last_n_turns=1,  # Use the last turn for the state of the music.
    system_instruction=SystemPrompt(
        contents="You can start the music, change the genre, volume, and equalizer settings."
    ),
)

# InMemoryConversationMemory

In [9]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=InMemoryConversationMemory(),
    planning=True,
)

In [10]:
prompt = "Turn this place into a lively party mood!"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:01:00] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:01:00] INFO | root: Generating text


[2025-01-10 00:01:04] DEBUG | Response: [TextResponse(text='### Plan to Turn the Place into a Lively Party Mood:\n\n1. **Lighting Setup**:\n   - Use `lights_agent` to turn the lights on.\n   - Adjust the brightness of the lights to a dimmer, party-appropriate level using `adjust_lights`.\n\n2. **Disco Ball Activation**:\n   - Use `disco_ball_agent` to power on the spinning disco ball for a fun and lively atmosphere.\n\n3. **Music Setup**:\n   - Use `music_agent.music_control_agent` to start playing upbeat party music in a suitable genre (e.g., pop, dance, or electronic).\n   - Adjust the music volume to a lively but comfortable level using `change_music_volume`.\n   - Optionally, use `music_agent.equalizer_control_agent` to tweak the equalizer settings for enhanced bass and treble, creating a more dynamic sound.\n\n4. **Final Adjustments**:\n   - Make any necessary changes to the music or lighting based on the vibe of the party.\n\nWould you like me to execute this plan?')]
[2025-01-10

In [11]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.8,
  'bpm': 120,
  'equalizer_settings': {'hz_100': 6,
   'hz_200': 6,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 6,
   'hz_6400': 6}},
 'lights': {'brightness': 0.7}}

In [12]:
list(response.usage.items())

[('equalizer_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=4527, output_tokens=262)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=4344, output_tokens=475)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=3982, output_tokens=545)),
 ('lights_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=799, output_tokens=35)),
 ('music_control_agent',
  Usage(model_name='gemini-1.5-flash-002', prompt_tokens=2006, output_tokens=180)),
 ('disco_ball_agent',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=589, output_tokens=49))]

Next turn prompt

In [13]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:03:05] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:03:05] INFO | root: Generating text


[2025-01-10 00:03:07] DEBUG | Response: [TextResponse(text='### Plan to Answer the Question:\n1. Use the `music_agent.music_control_agent` to retrieve the current music settings, including the genre, volume, and any other relevant details.\n2. If the tool does not provide the required information directly, check the conversation history for previously set music settings.\n3. Provide a concise summary of the current music settings based on the retrieved or historical data.\n\nWould you like me to proceed?')]
[2025-01-10 00:03:07] DEBUG | Prompt: [TextPrompt(text='Put the plan into action.')]
[2025-01-10 00:03:07] INFO | root: Generating text
[2025-01-10 00:03:10] DEBUG | Response: [ToolCallResponse(name='final_answer', args='{"disco_ball":{"power":true},"music":{"genre":"pop","volume":0.8,"bpm":120,"equalizer_settings":{"hz_100":6,"hz_200":6,"hz_400":0,"hz_800":0,"hz_1600":0,"hz_3200":6,"hz_6400":6}},"lights":{"brightness":0.7}}', call_id='call_ByermW0HxliY9NvOnAyhcjO2')]
[2025-01-10 00

In [14]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.8,
  'bpm': 120,
  'equalizer_settings': {'hz_100': 6,
   'hz_200': 6,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 6,
   'hz_6400': 6}},
 'lights': {'brightness': 0.7}}

Load history.

In [15]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

If you want to input conversation history while you don't want to store the response into the history, please set `store_conversation` as `False` in the `AgentConfig`.

In [21]:
from langrila import AgentConfig

# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=InMemoryConversationMemory(),
    planning=True,
    agent_config=AgentConfig(store_conversation=False),  # Disable storing conversation history
)

# PickleConversationMemory

`PickleConversationMemory` keeps the history as the local pickle file. The history of sub-agents is not persisted if you didn't pass an any conversation memory to sub-agents, but if you gave individual conversation memory to each sub-agents, you can persist all the history including sub-agent's history. 

In [16]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=PickleConversationMemory("./conversation.pkl"),
    planning=True,
)

In [17]:
prompt = "Turn this place into a lively party mood!"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:03:58] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:03:58] INFO | root: Generating text
[2025

In [18]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:04:33] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:04:33] INFO | root: Generating text
[2025-01-10 00

In [19]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.7,
  'bpm': 120,
  'equalizer_settings': {'hz_100': 5,
   'hz_200': 5,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 5,
   'hz_6400': 5}},
 'lights': {'brightness': 0.8}}

Load history

In [20]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

The conversation history within a sub-agent is automatically stored in memory even if you don't do anything but it's not persisted. Ofcourse you can persist all the conversation history including sub-agent's one if you specify conversation memory for each sub-agent.

# JSONConversationMemory

`JSONConversationMemory` stores the history into the local json file.

In [21]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=JSONConversationMemory("./conversation.json"),
    planning=True,
)

In [22]:
prompt = "Turn this place into a lively party mood!"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:05:37] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:05:37] INFO | root: Generating text
[2025

In [23]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:06:25] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:06:25] INFO | root: Generating text
[2025-01-10 00

In [24]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.7,
  'bpm': 110,
  'equalizer_settings': {'hz_100': 6,
   'hz_200': 6,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 6,
   'hz_6400': 6}},
 'lights': {'brightness': 0.8}}

Load history

In [25]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

# S3ConversationMemory

`S3ConversationMemory` allows you to store the history into the AWS S3 bucket.

Acknowledgement: [@kun432](https://github.com/kun432)

In [26]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=S3ConversationMemory(
        bucket="langrila-test",
        object_key="sample_conversation.json",  # "PREFIX/OBJECT_KEY" for using prefix
        aws_access_key_id_env_name="AWS_ACCESS_KEY",  # # env variable name, as needed
        aws_secret_access_key_env_name="AWS_SECRET_KEY",  # # env variable name, as needed
    ),
    planning=True,
)

In [27]:
prompt = "Turn this place into a lively party mood!"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:06:56] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:06:56] INFO | root: Generating text
[2025

In [28]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:07:31] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:07:31] INFO | root: Generating text
[2025-01-10 00

In [29]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.7,
  'bpm': 110,
  'equalizer_settings': {'hz_100': 6,
   'hz_200': 6,
   'hz_400': 0,
   'hz_800': 3,
   'hz_1600': 3,
   'hz_3200': 6,
   'hz_6400': 6}},
 'lights': {'brightness': 0.8}}

In [30]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a

# CosmosConversationMemory

`CosmosConversationMemory` is the module to store history to the Azure Cosmos DB. Please create Cosmos DB for your azure subscription in advance.

Acknowledgement: [@rioriost](https://github.com/rioriost) for base implementation.

In [31]:
# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    response_schema_as_tool=ResponseSchema,
    conversation_memory=CosmosConversationMemory(
        endpoint_env_name="COSMOS_ENDPOINT",  # env variable name
        key_env_name="COSMOS_KEY",  # env variable name
        db_env_name="COSMOS_DB_NAME",  # env variable name
        container_name="test",
        item_name="sample_conversation",
    ),
    planning=True,
)

In [32]:
prompt = "Turn this place into a lively party mood!"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:08:02] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:08:02] INFO | root: Generating text
[2025

In [33]:
prompt = "What the current music settings?"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-10 00:08:38] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nWhat the current music settings?\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')]
[2025-01-10 00:08:38] INFO | root: Generating text
[2025-01-10 00

In [34]:
ResponseSchema.model_validate_json(response.contents[0].text).model_dump()

{'disco_ball': {'power': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>,
  'volume': 0.8,
  'bpm': 110,
  'equalizer_settings': {'hz_100': 6,
   'hz_200': 6,
   'hz_400': 0,
   'hz_800': 0,
   'hz_1600': 0,
   'hz_3200': 6,
   'hz_6400': 6}},
 'lights': {'brightness': 0.8}}

In [35]:
orchestrator.load_history()

[[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a lively party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball on or off.\n- music_agent\n  - music_control_agent\n    - start_music: Start playing a random music with the given the genre.\n    - change_music: Change the music.\n    - change_music_volume: Change the music volume.\n  - equalizer_control_agent\n    - change_equalizer_settings: Change the equalizer settings of the music.\n')], name=None),
  Response(type='Response', role='a